# Data Collection
## Setup

In [ ]:
import serial
import sys
import glob
import re
import numpy
import pandas
from matplotlib import pyplot
from IPython.display import display, clear_output
from pathlib import Path
import datetime

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

print(serial_ports())

# >>>>>>>>> User Input <<<<<<<<<
# Select COM port
com_port = "COM3" # Change this to the correct port for the Arduino (see print below)
timeout = 5
default_array_size = 1e6
data_base_path = Path("C:/Users/iSenSys/Desktop/Hongyu_Capacitive_Testing/testresults")

# Get connection
ser = serial.Serial(com_port, timeout=timeout)

# Compile regex
re_pattern = re.compile(r"Time: (?P<Time>\d+.\d+) seconds  Capacitance: (?P<Capacitance>\d+.\d+) pF")  # < UNCOMMENT me when C code changed
#re_pattern = re.compile(r"Capacitance: (?P<cap>\d+.\d+) pF")

['COM1', 'COM3', 'COM5']


# Collect

Keep clicking play until it works :)

Press stop when you're done with testing and ready to store the collected data to a file

In [ ]:
# Performance optimizations
# pyplot.ioff()

# Create a figure
fig, ax = pyplot.subplots(figsize=(10, 2), dpi=200)
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Capacitance (pF)")

# Preallocate array memory
array = numpy.empty((int(default_array_size), 2))
array[:,:] = numpy.nan

# Collect data
try:
    idx = 0
    while True:
        # Collect data and extract values
        line = ser.readline()
        match = re_pattern.match(line.decode('utf-8'))
        print(line) #Display current reading (plotting too slow)

        # Store data
        array[idx, 0] = float(match.group("Time"))
        array[idx, 1] = float(match.group("Capacitance"))
        idx += 1

        # Plot it like its hot
        """
        ax.plot(array[:, 0], array[:, 1], color="blue")
        display(fig)
        clear_output(wait=True)
        """

except KeyboardInterrupt:
    pass # expected -> what happens when you stop the cell

## Save Data

Press start to save all of the data collected from the previous step to a .csv

In [ ]:
# Create dataframe
data_df = pandas.DataFrame(array, columns=['Time','Capacitance'])  # CHANGE ME: to include time
data_df = data_df.dropna()

# Construct file name
now_time = datetime.datetime.now().isoformat(timespec="seconds")
now_time = now_time.replace(":", "-")
data_file_name = "cap_data_" + now_time + ".csv"

# Save
data_df.to_csv(data_base_path / data_file_name)

## Shutdown Communications

Must run this or restart kernel to release COM3

In [ ]:
ser.close()